In [2]:
# Read all the ERROR files and give a stats.
import sys
import os
import json

def read_file(filename):
  with open(filename, 'r') as f:
    return f.read()

def read_file_none(filename):
  try: return read_file(filename)
  except: return None

def write_file(filename, content):
  with open(filename, 'w') as f:
    f.write(content)

def read_json(filename):
  return json.loads(read_file(filename))

def read_json_none(filename):
  try: return read_json(filename)
  except: return None

def write_json(filename, content):
  write_file(filename, json.dumps(content, indent=2))

scenario_to_infix = {
  "basecodex": "buggy",
  "basefixed": "fixed",
  "basecomment": "comments",
  "basevar": "renamed",
  "group2fixed": "multi"
}

CODEMAP_PATH = "../../data/transmap/codemap"

def get_stats_for_scenario(scenario):
  default_log_file = os.path.join(CODEMAP_PATH, f"./cm_3_counterr.{scenario}.ERROR.log")
  manual_log_file = os.path.join(CODEMAP_PATH, f"./cm_3_counterr.{scenario}.ERRMANUAL.log")
  log_content = read_file_none(manual_log_file)
  if log_content is None: 
    print("% Warning: cannot find manual log file, use default log file")
    log_content = read_file_none(default_log_file)
    if log_content is None: raise Exception(f"Cannot find {default_log_file}")
  lines = log_content.strip().split("\n")
  # next is to compute the stats. First ignore lines started with "#"
  lines = [line for line in lines if not line.startswith("#")]
  macro_infix = scenario_to_infix[scenario]
  # latex macros
  macros = []
  # --- accuracy `\cmacc{macro_infix}` is defined as the percentage of 292 - len(lines) / (292)
  percentage = round((292 - len(lines)) / 292 * 100, 1)
  macros.append(f"\\newcommand{{\\cmacc{macro_infix}}}{{{percentage}}}")
  # --- neq count `\cm{macro_infix}neq` is the number of lines that contains `Expected str equal, got`
  neq_lines = [line for line in lines if "Expected str equal, got" in line]
  neq_count = len(neq_lines)
  neq_percentage = round(neq_count / 292 * 100, 1)
  macros.append(f"\\newcommand{{\\cm{macro_infix}neq}}{{{neq_percentage}}}")
  # --- oob count `\cm{macro_infix}oob` is the number of lines that contains `js has INVALID out-of-range idxes`
  oob_lines = [line for line in lines if "js has INVALID out-of-range idxes" in line]
  oob_count = len(oob_lines)
  oob_percentage = round(oob_count / 292 * 100, 1)
  macros.append(f"\\newcommand{{\\cm{macro_infix}oob}}{{{oob_percentage}}}")
  # --- diso count `\cm{macro_infix}diso` is the number of lines that contains `js NOT MONOTONOUS`
  diso_lines = [line for line in lines if "js NOT MONOTONOUS" in line or "[UNMAP]" in line or "[DISO]" in line]
  diso_count = len(diso_lines)
  diso_percentage = round(diso_count / 292 * 100, 1)
  macros.append(f"\\newcommand{{\\cm{macro_infix}diso}}{{{diso_percentage}}}")
  # lines that does not belong to any of the above
  other_lines = [line for line in lines if line not in neq_lines and line not in oob_lines and line not in diso_lines]
  other_count = len(other_lines)
  other_percentage = round(other_count / 292 * 100, 1)
  macros.append(f"\\newcommand{{\\cm{macro_infix}other}}{{{other_percentage}}}")

  print("\n".join(macros))
  print("\n".join([" %" + x for x in other_lines]))
  print("")

for scenario in scenario_to_infix:
  get_stats_for_scenario(scenario)

\newcommand{\cmaccbuggy}{92.5}
\newcommand{\cmbuggyneq}{3.8}
\newcommand{\cmbuggyoob}{1.7}
\newcommand{\cmbuggydiso}{2.1}
\newcommand{\cmbuggyother}{0.0}


\newcommand{\cmaccfixed}{94.5}
\newcommand{\cmfixedneq}{2.1}
\newcommand{\cmfixedoob}{2.1}
\newcommand{\cmfixeddiso}{1.4}
\newcommand{\cmfixedother}{0.0}


\newcommand{\cmacccomments}{93.8}
\newcommand{\cmcommentsneq}{1.4}
\newcommand{\cmcommentsoob}{2.7}
\newcommand{\cmcommentsdiso}{2.1}
\newcommand{\cmcommentsother}{0.0}


\newcommand{\cmaccrenamed}{91.4}
\newcommand{\cmrenamedneq}{2.1}
\newcommand{\cmrenamedoob}{5.8}
\newcommand{\cmrenameddiso}{0.7}
\newcommand{\cmrenamedother}{0.0}


\newcommand{\cmaccmulti}{89.4}
\newcommand{\cmmultineq}{4.8}
\newcommand{\cmmultioob}{3.1}
\newcommand{\cmmultidiso}{2.7}
\newcommand{\cmmultiother}{0.0}


